In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import mysql.connector as mysqlpy
import os
from pathlib import Path

## Chargement des datas

In [2]:
def get_db_connection():
    user = os.environ.get('DB_USER')
    password = os.environ.get('DB_PASSWORD')
    host = os.environ.get('DB_HOST')
    port = os.environ.get('DB_PORT')
    database = os.environ.get('DB_DATABASE')
    bdd = mysqlpy.connect(user=user, password=password, host=host, port=port, database=database)
    return bdd

bdd = get_db_connection()
cursor = bdd.cursor()
cursor.execute(f'''SELECT CRO, id_diag, name, organe, diagnostic FROM CRO 
               JOIN patients USING (nir)
               JOIN diagnostics USING (id_diag);''')
CRO = cursor.fetchall()

columns=[i[0] for i in cursor.description]

CRO = pd.DataFrame(CRO, columns= columns)

cursor.execute(f'''SELECT id_diag, diagnostic, organe FROM diagnostics;''')
diag = cursor.fetchall()

columns=[i[0] for i in cursor.description]

diag = pd.DataFrame(diag, columns= columns)

cursor.close()

True

In [3]:
CRO.head(5)

,CRO,id_diag,name,organe,diagnostic
0,Résumé anatomo-pathologique\nPatient : Gilbert...,5,Gilbert Étienne,peau,lichen plan
1,Résumé opératoire d'Histopathologie :\nPatient...,2,Gilbert Étienne,peau,mélanome
2,Résumé anatomopathologique\nPatient : Gilbert ...,20,Gilbert Étienne,côlon,polypose adénomateuse familiale
3,Résumé anatomopathologique\nPatient : Gilbert ...,2,Gilbert Étienne,peau,mélanome
4,Résumé anatomopathologique\nPatient : Joseph B...,16,Joseph Berthelot,côlon,tumeur neuroendocrine


In [4]:
diag.head(5)

,id_diag,diagnostic,organe
0,1,psoriasis,peau
1,2,mélanome,peau
2,3,rosacée,peau
3,4,lupus érythémateux,peau
4,5,lichen plan,peau


In [13]:
def find_sentence_with_diag(texte: str, diag: str, patient: str) -> dict:  
    for rep in [f'chez {patient}', f'chez M {patient}', f'chez Mme {patient}', f'chez M {patient.split()[1]}', f'chez Mme {patient.split()[1]}']:
        texte = texte.replace(rep, 'chez le patient')
    for rep in [f'de {patient}', f"d'{patient}",  f'de M {patient}', f'de Mme {patient}', f'de M {patient.split()[1]}', f'de Mme {patient.split()[1]}']:
        texte = texte.replace(rep, 'du patient')
    for rep in [f'Le patient {patient}', f'le patient {patient}', f'la patient M {patient}', f'le patient M {patient.split()[1]}']:
        texte = texte.replace(rep, 'le patient')
    for rep in [f'sur {patient}', f'sur {patient.split()[1]}', f'sur M {patient}', f'sur M {patient.split()[1]}', f'sur Mme {patient}', f'sur Mme {patient.split()[1]}']:
        texte = texte.replace(rep, 'le patient')  
    text_split = texte.split('\n')

    occurrence = {}
    for sentence in text_split:
        index = -1
        while True:
            texte = sentence.lower()
            # trouve toutes les occurences d'un mot dans le texte jusqu'à la fin
            index = texte.find(diag.lower(), index + 1)
            if index == -1:
                break

            index_fin = index + len(diag)

            # vérifie par quoi se termine le mot pour éviter les mots dérivés du mot recherché
            if texte[index_fin] in [' ', ',', '.', ')', ':']:                
                occurrence = {'sentence': sentence, 'diag': diag}
            else:
                print("mot suivi d'une lettre", diag, '\n', sentence[index:])
    return occurrence

def find_sentence_without_diag(texte: str, diag: str) -> list:
    text_split = texte.split('\n')
    for paragraphe in text_split:
        if diag.lower() not in paragraphe.lower():
             if len(paragraphe) >= 200:
                return paragraphe

def sentence_version(dict_sentences: dict) -> list:
        sentences_list = dict_sentences['sentence']
        sentences_list = sentences_list.split('.')
        for sentence in sentences_list:
             if dict_sentences['diag'] in sentence:
                sentence_split = sentence.split(dict_sentences['diag'])
                sentence_before_diag = sentence_split[0] 
                sentence_after_diag = sentence_split[1]
                sentence_list = sentence_before_diag.split() 
                if sentence_list[len(sentence_list) -1] == 'la' or sentence_list[len(sentence_list) -1] == 'le' or sentence_list[len(sentence_list) -1] == 'un' or sentence_list[len(sentence_list) -1] == 'une':
                    sentence_list = sentence_list[0: len(sentence_list) - 1]
                    sentence_before_diag = " ".join(sentence_list) + " un "
                if sentence_list[len(sentence_list) -1] == "d'un" or sentence_list[len(sentence_list) -1] == "d'une":
                    sentence_list = sentence_list[0: len(sentence_list) - 1]
                    sentence = " ".join(sentence_list) + " d'un "
                sentence_before_diag = sentence_before_diag + '[[DIAG]]'                 

                return sentence_before_diag, sentence_after_diag
        return "", ""


In [26]:
def generate_list_of_sentences_diag(CRO: pd.DataFrame, diag: pd.DataFrame, data: str) -> list:
    random.seed(0)
    
    # stocke les paragraphes contenant les diagnostics et les diagnostics présents dans le CRO
    paragraphes = []
    paragraphes_sans_diag = []
    for n in range(0, len(CRO)):
        paragraphes.append(find_sentence_with_diag(CRO.CRO[n], CRO.diagnostic[n], CRO.name[n]))
        paragraphes_sans_diag.append(find_sentence_without_diag(CRO.CRO[n], CRO.diagnostic[n]))
    # stocke les phrases qui contient la diagnostic en vérifiant leur absence au préalable
    sentences_before_diag = []
    sentences_after_diag = []
    
    for resp in paragraphes:
        sentence_before_diag, sentence_after_diag = sentence_version(resp)
        if sentence_before_diag not in sentences_before_diag:                          
            sentences_before_diag.append(sentence_before_diag)        
        sentences_after_diag.append(sentence_after_diag)
    random.shuffle(sentences_after_diag)

    # génère une liste de diagnostics de la même longueur que le nombre de phrase unique repérée
    symbole = '=' if data == 'test' else '!'
    diags_list = pd.read_csv('..\data\diagnostics_extend.csv', sep=';')
    diags_list = diags_list.query(f'organe {symbole}= "foie"')

    diags_list = diags_list.diagnostic.unique().tolist()
    nb_sentences = len(sentences_before_diag)
    facteur_mulitplicateur = nb_sentences // len(diags_list)
    diags_list_multi = diags_list * facteur_mulitplicateur
    random.shuffle(diags_list_multi)
    diags_list_complement = diags_list
    random.shuffle(diags_list_complement)
    manque = len(sentences_before_diag) - len(diags_list_multi)
    diags_list_multi = diags_list_multi + diags_list_complement[0: manque]

    # remplace les diagnostics présents dans les phrases retenues
    sentences_with_diags = []
    for n in range(0, len(sentences_before_diag)):
        sentence = sentences_before_diag[n].replace('[[DIAG]]', diags_list_multi[n])
        sentences_with_diags.append({'text' : sentence, 'diag': diags_list_multi[n]})

    # remplace les phrases contenants les diagnostics dans les paragraphes retenus
    for sentences in paragraphes:
        sentences_list = sentences['sentence'].split('.')
        for n in range(0, len(sentences_list)):
            if sentences['diag'] in sentences_list[n]:
                sentences_list[n] = ' [[SENTENCE]]'
        sentences['sentence'] = '.'.join(sentences_list)

    random.shuffle(paragraphes)

    # remplace les phrases présent initialement dans le paragraphe par une phrase unique
    new_paragraphes = []
    for n in range(0, len(sentences_with_diags)):
        sentences = paragraphes[n]['sentence'].strip()
        diag = sentences_with_diags[n]['diag']
        sentences = sentences.replace('[[SENTENCE]]', sentences_with_diags[n]['text'])
        index_debut = sentences.find(diag)
        index_fin = index_debut + len(diag)
        new_paragraphes.append([sentences, {'entities': [[index_debut, index_fin, 'DIAG']]}])
    
    if data != 'test':
        nb_paragraphes_without_diag = int(len(new_paragraphes) * 10 / 100)
        print(len(paragraphes_sans_diag))
        random.shuffle(paragraphes_sans_diag)
        paragraphes_sans_diag = paragraphes_sans_diag[0 : nb_paragraphes_without_diag - 1]
        for paragraphe in paragraphes_sans_diag:
            if paragraphe is not None:
                new_paragraphes.append([paragraphe, {'entities': []}])

    random.shuffle(new_paragraphes)

    return new_paragraphes

def csv_to_json(list_of_sentences: list, file_json : Path) -> str:
    '''
    Génération d'un fichier JSON qui rassemble le texte, les index de début et de fin de chaque mot associé à leur label
    - entrées : 
    - sortie : un fichier JSON
    '''
    # ouverture du fichier JSON
    to_json = json.dumps(list_of_sentences, ensure_ascii=False)

    with open(file_json, 'w', encoding="utf-8") as fichier:
        # écriture du fichier JSON, ajout si existe déjà !
        fichier.write(to_json)
        

In [27]:
CRO_train_valid = CRO.query('organe != "foie"').reset_index(drop=True)
CRO_test = CRO.query('organe == "foie"').reset_index(drop=True)

CRO_train, CRO_valid = train_test_split(CRO_train_valid, test_size=0.2, stratify=CRO_train_valid['id_diag'], random_state=42)
CRO_train= CRO_train.reset_index(drop=True)
CRO_valid = CRO_valid.reset_index(drop=True)

train_data = generate_list_of_sentences_diag(CRO_train, diag, 'train')
valid_data = generate_list_of_sentences_diag(CRO_valid, diag, 'valid')
test_data= generate_list_of_sentences_diag(CRO_test, diag, 'test')

print('génération du fichier json train')
taille_json_train = csv_to_json(train_data, '../generate_model/assets/train.json')
print(f'La data train possède {len(train_data)}')

print('génération du fichier json valid')
taille_json_valid = csv_to_json(valid_data, '../generate_model/assets/dev.json')
print(f'La data valid possède {len(valid_data)}')

print('génération du fichier json test')
taille_json_test = csv_to_json(test_data, '../generate_model/assets/test.json')
print(f'La data test possède {len(test_data)}')

mot suivi d'une lettre diverticulite 
 diverticulites. Il s'agissait principalement de diverticules simplex localisés aux bords du spécimen. Des zones d'inflammation aiguë avec infiltrat polynucléaire neutrophile (PN) abondant et dégranulation des mastocytes ont été observées. Les cryptes intestinales restaient intactes. Aucune atteinte dysplasique ou cancerienne n'a été identifiée.
1712
mot suivi d'une lettre diverticulite 
 diverticulites de suivre une alimentation riche en fibres pour prévenir la formation de complications telles qu'une perforation ou une infection. Un contrôle régulier devrait être considéré pour surveiller l'évolution de cette pathologie.
428
mot suivi d'une lettre cholangiocarcinome 
 cholangiocarcinomes.
mot suivi d'une lettre hépatoblastome 
 hépatoblastomes. Des foci de différentiation hépatocytaires sont observés dans certaines sections, avec des cellules hépatiques matures et des biles canaliculaires bien formées. Cependant, des zones ne montrent pas de diff

: 